In [ ]:
sc

In [ ]:
from os import getcwd, listdir
from os.path import join, isfile
from time import time
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
conn_str = "jdbc:mysql://localhost/test_tpcds"

In [ ]:
spark = SparkSession.builder.master("local[1]").appName("JAS").enableHiveSupport().getOrCreate()

In [ ]:
tables = ["call_center", "catalog_page", "catalog_returns", "catalog_sales", "customer", "customer_address", "customer_demographics", "date_dim", "dbgen_version", "household_demographics", "income_band", "inventory", "item", "promotion", "reason", "ship_mode", "store", "store_returns", "store_sales", "time_dim", "warehouse", "web_page", "web_returns", "web_sales", "web_site"]

In [ ]:
schema_dict = {}
df_dict = {}

for index, table in enumerate(tables):
    print("{}: '{}' table\n".format( index + 1, table ))
    table_df = spark.read.format("jdbc").option("driver", "com.mysql.jdbc.Driver").option("url", conn_str).option("dbtable", table).option("user", "root").load()
    schema_dict[table] = table_df.schema
    df = spark.read.load( join(getcwd(), "data_correct", "{}.csv".format( table )),
        format="csv",
        delimiter="|",
        header="true",
        schema=table_df.schema)
    df_dict[table] = df
    df.createOrReplaceTempView("{}".format( table ))

In [18]:
sql_queies_path = join(getcwd(), "sql_queries")
listdir(sql_queies_path)

['query_0.sql', 'query_test.sql', '.ipynb_checkpoints']

In [ ]:
for query_file in listdir(sql_queies_path):
    file_path = join(sql_queies_path, query_file)
    if not isfile(file_path): continue
    with open(file_path, "rt") as file:
        sql_code = file.read()
        spark.sql(sql_code).show()

##### Make Separate files

In [27]:
with open(join(sql_queies_path, "tpcds_queries.sql"), "rt") as file:
    lines = file.readlines()
    query = ""
    query_no = 1
    for index, line in enumerate(lines):
        query_line = line.strip()
        query += "\n{}".format( query_line )
        
        # Separate each query
        if query_line.find("-- end query", 0, len(query_line)) != -1:
            
            with open(join(sql_queies_path, "query_{}.sql".format(query_no)), "w") as query_file:
                query_file.write(query)
                query = ""
                print("Query - {} written.".format(query_no))
                query_no = query_no + 1


Query - 1 written.
Query - 2 written.
Query - 3 written.
Query - 4 written.
Query - 5 written.
Query - 6 written.
Query - 7 written.
Query - 8 written.
Query - 9 written.
Query - 10 written.
Query - 11 written.
Query - 12 written.
Query - 13 written.
Query - 14 written.
Query - 15 written.
Query - 16 written.
Query - 17 written.
Query - 18 written.
Query - 19 written.
Query - 20 written.
Query - 21 written.
Query - 22 written.
Query - 23 written.
Query - 24 written.
Query - 25 written.
Query - 26 written.
Query - 27 written.
Query - 28 written.
Query - 29 written.
Query - 30 written.
Query - 31 written.
Query - 32 written.
Query - 33 written.
Query - 34 written.
Query - 35 written.
Query - 36 written.
Query - 37 written.
Query - 38 written.
Query - 39 written.
Query - 40 written.
Query - 41 written.
Query - 42 written.
Query - 43 written.
Query - 44 written.
Query - 45 written.
Query - 46 written.
Query - 47 written.
Query - 48 written.
Query - 49 written.
Query - 50 written.
Query - 5

##### Do the benchmarking

In [28]:
take_avg_run, skip_run = 5, 1
total_runs = take_avg_run + skip_run

In [42]:
def get_execute_query_time(file):
    with open(file, "rt") as f:
        try:
            start_time = time()
            spark.sql(f.read())
            return time() - start_time      
        except:
            print("[[ error ]]: Failed to execute '{}' query :(".format( file ))
            return None

In [43]:
timings_dict = {}
total_queries = sorted([x for x in listdir(sql_queies_path) if str(x).startswith("query_")])
# print(total_queries)

for index, query in enumerate(total_queries):
    print("="*50)
    print("{} - Running benchmark for query: {}".format(index + 1, query))
    
    timings = list()
    query_file = join(sql_queies_path, query)

    for run in list(range(0, total_runs)):
        executed_time = get_execute_query_time(query_file)
        if executed_time is None: continue
        if run == 0:
            print("Skipping 1st run")
        else:
            timings.append(executed_time)
            print("{} run took {} secs".format(run, executed_time))
    timings_dict[query] = timings

1 - Running benchmark for query: query_1.sql
[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/sql_queries/query_1.sql' query :(
[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/sql_queries/query_1.sql' query :(
[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/sql_queries/query_1.sql' query :(
[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/sql_queries/query_1.sql' query :(
[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/sql_queries/query_1.sql' query :(
[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/sql_queries/query_1.sql' query :(
2 - Running benchmark for query: query_10.sql
[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data Warehouse/tpcds-kit/sql_queries/query_10.sql' query :(
[[ error ]]:

In [60]:
i = int(0)
for key, value in timings_dict.items():
    if len(value):
        i = i + 1
        print("="*20)
        print("\n{} - {}: {} secs\n".format(i, key, sum(value)/len(value)))
        # print("Avg.: {}".format( sum(value)/len(value) ))


1 - query_13.sql: 0.02523198127746582 secs


2 - query_20.sql: 0.17638463973999025 secs


3 - query_49.sql: 0.015842294692993163 secs


4 - query_50.sql: 0.07642388343811035 secs


5 - query_66.sql: 0.030879926681518555 secs


6 - query_72.sql: 0.019758176803588868 secs


7 - query_73.sql: 0.01715097427368164 secs


8 - query_74.sql: 0.016375350952148437 secs


9 - query_77.sql: 0.010669708251953125 secs


10 - query_79.sql: 0.013879060745239258 secs


11 - query_84.sql: 0.026879405975341795 secs


12 - query_91.sql: 0.018711280822753907 secs

